In [55]:
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from file_functions import File

FILEPATH = r"C:\Users\ASUS\OneDrive\Desktop\code\datasets\protein.csv"

# @tool
# def information(file_path):
#     'returns file information'
#     val=File(file_location=file_path)
#     return val.discribe_df()

# tools=[information]
# toolnode=ToolNode(tools=tools)

In [56]:
from langchain_community.llms import Ollama
from langchain.agents import initialize_agent, Tool
llm = Ollama(model="llama3.2")


In [57]:
from langchain_core.messages import BaseMessage,ToolMessage,SystemMessage,HumanMessage,AIMessage
from langgraph.graph import StateGraph, END ,MessagesState, START
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
import re
from mcp.client.session import ClientSession
import anyio

async def call_information(path: str):
    async with ClientSession("CSVToolServer") as session:
        tools = await session.list_tools()
        print("Tools available:", tools)
        response = await session.call_tool("information", {"file_path": path})
        return response
    
def choose(state:MessagesState):
    # a=input('Query')
    a='analysis'
    global FILEPATH
    if 'path' in a or 'save' in a:
        match = re.search(r"[A-Z]:\\(?:[^\\/:*?\"<>|\r\n]+\\)*[^\\/:*?\"<>|\r\n]+", a)
        if match:
            FILEPATH = match.group()
            return 'path'
    
    if 'exit' == a.lower():
        return "end"

    if 'analysis' in a.lower():
        return 'analysis'

async def analysis(state:MessagesState):
    global FILEPATH
    file_info = await call_information(FILEPATH)
    systemprompt=[SystemMessage(content='You are a data analyst. you are provided infromatoion about a csv file now use that information to make a report.'),
    HumanMessage(content=str(file_info))]
    response=llm.invoke(systemprompt)
    print(response)
    return {"messages": state['messages']+[response]}




In [58]:
graph = StateGraph(MessagesState)
graph.add_node('choose',lambda state:state)
graph.add_node('analysis',analysis)
graph.set_entry_point('choose')
graph.add_conditional_edges(
    'choose',
    choose,
    {
        'end':END,
        'analysis':'analysis',
        'path':'choose',
    }
)
graph.add_edge('analysis','choose')
app = graph.compile()

In [59]:
from IPython.display import Image,display
display(Image(app.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [60]:
res = app.invoke({"messages": ["blablabla"]})
print(res['messages'])

TypeError: No synchronous function provided to "analysis".
Either initialize with a synchronous function or invoke via the async API (ainvoke, astream, etc.)